# Mapping và Chuẩn hóa Entity Names (Major)

Notebook này thực hiện mapping `entityName` → `entityNameNormalized` cho bảng `Post_Entity` dựa trên logic từ `normalize_majors.py`.

## 1. Import Libraries và Khởi tạo Spark Session

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, trim, lower, regexp_replace
from pyspark.sql.types import StringType
import os
import re

# Set AWS environment variables for MinIO
os.environ['AWS_REGION'] = 'us-east-1'
os.environ['AWS_ACCESS_KEY_ID'] = 'admin'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'admin123'

# Khởi tạo Spark Session với Iceberg và Nessie catalog
spark = (
    SparkSession.builder.appName("Mapping_Post_Entity_MAJ")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "1536m")
    .config("spark.executor.cores", "2")
    # ===== Iceberg Catalog qua Nessie =====
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://gold/")
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    # ===== Cấu hình MinIO =====
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.sql.catalog.nessie.s3.region", "us-east-1")
    # ===== Spark + Hadoop S3 connector =====
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")
    .config("spark.executorEnv.AWS_REGION", "us-east-1")
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", "admin")
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", "admin123")
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
print("✓ Spark Session đã được khởi tạo với Nessie catalog!")
print(f"Spark Master: {spark.sparkContext.master}")
print(f"Application ID: {spark.sparkContext.applicationId}")

## 2. Định nghĩa Major Mapping Dictionary

Dictionary này chứa các quy luật mapping ngành học, tương tự như `normalize_majors.py`

## 3. Định nghĩa Hàm Normalize và Mapping

In [ ]:
def normalize_text(text):
    """Chuẩn hóa text: lowercase, loại bỏ prefix NHIỀU LẦN"""
    import re
    if not text:
        return ""
    
    text = str(text).lower().strip()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'["""'']', '', text)
    
    # Loại bỏ prefix NHIỀU LẦN (cho đến khi không còn)
    prefixes = ['ngành ', 'nghề ', 'học ', 'sv ', 'chuyên ngành ', 'chuyên viên ']
    changed = True
    max_iter = 5
    iteration = 0
    
    while changed and iteration < max_iter:
        changed = False
        for prefix in prefixes:
            if text.startswith(prefix):
                text = text[len(prefix):].strip()
                changed = True
                break
        iteration += 1
    
    return text

def find_major_group(normalized_name):
    """Tìm nhóm ngành - Dictionary ĐÃ LÀM SẠCH
    Trả về key nếu tìm thấy, None nếu không tìm thấy trong dictionary
    """
    
    # Dictionary đã làm sạch - 148 nhóm ngành (đã xóa 9 nhóm không hợp lý)
    major_mapping = {
        # ===== CÔNG NGHỆ THÔNG TIN =====
        'công nghệ thông tin': ['công nghệ thông tin', 'congnghethongtin', 'cntt', 'it', 'công nghệ it', 'kỹ sư công nghệ thông tin', 'cto', 'khoa học công nghệ', 'khối công nghệ', 'mạng máy tính', 'tin học', 'tin học ứng dụng', 'công nghệ máy tính', 'công nghệ số', 'hệ thống máy tính mạng dữ liệu', 'máy tính và dữ liệu', 'nghiên cứu máy tính / thông tin', 'mobile app developer', 'kiểm thử', 'qa', 'attt', 'cn tt', '5g', 'it.giờ', 'lắp ráp máy tính', 'lập trình nhúng', 'embedded', 'ngành hệ thống thông tin quản lý', 'hệ thống thông tin', 'hệ thống thông tin quản lý', 'tư vấn erp', 'quản lý thông tin'],
        'phát triển phần mềm': ['phần mềm', 'software', 'kỹ thuật phần mềm', 'lập trình viên', 'lập trình', 'full-stack', 'frontend', 'backend', 'công nghệ phần mềm', 'ứng dụng phần mềm', 'phát triển website', 'nhà phát triển web', 'solution architect', 'technical product manager', 'php'],
        'khoa học dữ liệu': ['data science', 'khoa học dữ liệu', 'big data', 'phân tích dữ liệu', 'big data engineer', 'data analyst', 'phân tích kinh doanh', 'dữ liệu lớn', 'chuyên gia dữ liệu lớn', 'data scientist', 'data analytic', 'data analytics', 'phân tích số liệu', 'nhân viên nhập liệu và xử lý dữ liệu', 'quản lý cơ sở dữ liệu', 'data analysis', 'data engineering - kỹ sư dữ liệu', 'ml engineer'],
        'trí tuệ nhân tạo': ['trí tuệ nhân tạo', 'ai', 'artificial intelligence', 'machine learning', 'trí thông minh nhân tạo'],
        'an ninh mạng': ['an ninh mạng', 'cyber security', 'bảo mật', 'bảo mật thông tin', 'an toàn thông tin', 'bảo vệ mạng'],
        'khoa học máy tính': ['khoa học máy tính', 'computer science', 'khmt', 'kỹ thuật máy tính', 'ktmt', 'computer engineering', 'kỹ thuật mạng', 'kỹ sư hệ thống', 'cloud engineer', 'kỹ sư siêu dữ liệu'],
        'devops': ['devops'],
        'iot': ['iot', 'internet of things'],
        'blockchain': ['blockchain', 'crypto'],
        'phát triển game': ['game', 'phát triển game', 'ngành game', 'thiết kế và phát triển game', 'sáng chế game'],
        
        # ===== KỸ THUẬT - CHẾ TẠO =====
        'cơ khí': ['cơ khí', 'kỹ thuật cơ khí', 'kỹ sư cơ khí', 'chế tạo máy', 'cnc', 'công nghệ kỹ thuật cơ khí', 'thợ hàn', 'nghề hàn', 'nghề hàn sắt', 'công nghệ chế tạo', 'công nghiệp chế tạo', 'chế tạo tinh xảo', 'khuôn mẫu đúc nhựa', 'chế biến nhựa', 'đúc', 'xử lý nhiệt', 'lò luyện kim', 'kỹ nghệ gỗ', 'thợ mộc', 'thủ công mỹ nghệ', 'đóng tàu', 'gia công – sản xuất – lắp ráp', 'công nhân sản xuất và lắp ráp', 'vận hành & sửa chữa máy công trình', 'sửa chữa', 'cad', 'kỹ thuật bào', 'công nhân vận hành máy móc', 'mechanical engineering', 'bảo trì – bảo dưỡng', 'bảo trì máy móc'],
        'điện - điện tử': ['điện', 'điện tử', 'điện công nghiệp', 'kỹ thuật điện', 'điện điện tử', 'điện – điện tử', 'điện lạnh'],
        'viễn thông': ['bưu chính viễn thông', 'viễn thám', 'viễn thông'],
        'tự động hoá': ['tự động hoá', 'tự động hóa', 'automation', 'cơ khí tự động hoá', 'điều khiển', 'robot', 'robotics', 'sản xuất tự động hoá', 'hệ thống tự hành', 'cơ điện tử', 'mechatronics', 'kỹ thuật cơ điện tử'],
        'ô tô': ['ô tô', 'kỹ thuật ô tô', 'công nghệ ô tô', 'công nghệ kỹ thuật ô tô', 'kĩ thuật ô tô'],
        'vi mạch - bán dẫn': ['vi mạch', 'bán dẫn', 'chip', 'thiết kế vi mạch', 'công nghệ bán dẫn'],
        'năng lượng tái tạo': ['năng lượng', 'năng lượng tái tạo', 'năng lượng sạch', 'năng lượng xanh', 'năng lượng bền vững', 'ngành chuyển đổi năng lượng và xanh', 'chuyên gia lưu trữ năng lượng', 'kỹ sư năng lượng tái tạo', 'công nghệ năng lượng', 'kĩ thuật năng lượng', 'năng lượng mới'],
        'kỹ thuật nhiệt': ['kỹ thuật nhiệt', 'công nghệ nhiệt', 'ngành kỹ thuật nhiệt', 'kỹ thuật máy lạnh và điều hoà không khí', 'nghề kỹ thuật máy lạnh và điều hoà không khí'],
        'quản lý công nghiệp': ['quản lý công nghiệp', 'kỹ thuật công nghiệp', 'ngành quản lí công nghiệp', 'hệ thống công nghiệp', 'quản lý sản xuất', 'kỹ sư sản xuất', 'công nghiệp chế biến', 'khoa học quản lý', 'ngành khoa học quản lý'],
        'khoa học vật liệu': ['vật liệu mới', 'cn vật liệu polyme & compozit', 'kỹ thuật vật liệu', 'công nghệ nano'],
        
        # ===== XÂY DỰNG =====
        'xây dựng': ['xây dựng', 'kỹ thuật xây dựng', 'kiến trúc', 'architect', 'quy hoạch', 'quy hoạch đô thị', 'quy hoạch vùng và đô thị', 'quản lý khu đô thị', 'quản lý đô thị', 'kết cấu', 'cấp thoát nước', 'xây lắp hạ tầng', 'cơ sở hạ tầng', 'trắc địa', 'thợ sửa ống nước', 'thợ lát gạch', 'kỹ sư cầu đường', 'civil engineering', 'kỹ thuật hiện trường', 'chuyên gia thiết kế đô thị', 'thiết kế cảnh quan'],
        'quản lý đất đai': ['quản lý đất đai', 'ngành quản lý đất đai', 'quản lí đất đai'],
        
        # ===== KINH TẾ - KINH DOANH =====
        'kinh tế': ['kinh tế', 'kinh tế học', 'qtkd', 'quản trị kinh doanh', 'kinhdoanh', 'kinh doanh', 'quản trị', 'chăm sóc khách hàng', 'nhân viên chăm sóc khách hàng', 'kd quốc tế', 'kt quốc tế', 'quản lý doanh nghiệp', 'quản lý thương hiệu', 'quản lý siêu thị', 'chuyên viên mua bán', 'buôn bán', 'giao dịch cơ bản', 'hỗ trợ giao dịch khách hàng', 'quan hệ khách hàng cá nhân / doanh nghiệp', 'dịch vụ khách hàng'],
        'sales': ['sales', 'sale', 'nhân viên kinh doanh', 'sales program', 'bán hàng', 'nhân viên bán hàng', 'nhân viên thu ngân', 'bán lẻ'],
        'tài chính - ngân hàng': ['tài chính', 'ngân hàng', 'tài chính ngân hàng', 'chứng khoán', 'tcnh', 'tc-nh', 'finance', 'cfo', 'banker', 'bảo hiểm', 'actuary', 'quản lý rủi ro', 'quản lý rủi ro và bảo hiểm', 'fintech', 'tài chính công nghệ', 'môi giới đầu tư', 'thu hồi vốn', 'sáp nhập doanh nghiệp', 'tài chính kiểm toán', 'phân tích tài chính', 'thẩm định giá', 'valuation', 'thẩm định giá trị', 'chuyên viên định giá tài sản'],
        'kế toán': ['kế toán', 'ketoan', 'kế kiểm', 'accountant', 'tư vấn thuế', 'thuế', 'công chức thuế'],
        'marketing': ['marketing', 'maketing', 'quản trị marketing', 'digital marketing', 'marketing số', 'seo', 'sem', 'ads', 'quảng cáo', 'marketer', 'digital mkt', 'tiếp thị', 'chuyên gia nghiên cứu thị trường', 'chuyên viên phân tích nghiên cứu thị trường', 'qc'],
        'nhân sự': ['quản lý nhân sự', 'quản trị nhân sự', 'nhân sự', 'hr', 'quản trị nhân lực', 'thư ký', 'văn phòng', 'văn thư', 'trợ lý ảo', 'ghi sổ', 'nhân viên ghi sổ', 'hành chính nhân sự'],
        'logistics': ['logistics', 'logistic', 'giao thông vận tải', 'quản lý chuỗi cung ứng', 'kho vận', 'đường sắt', 'tài xế xe tải', 'warehousing', 'đóng gói', 'xklđ'],
        'xuất nhập khẩu': ['xuất nhập khẩu', 'import export', 'thương mại quốc tế', 'chứng từ quốc tế', 'chuyên viên tài trợ thương mại', 'ttqt', 'ttđpt'],
        'quản lý dự án': ['quản lý dự án', 'project management'],
        'thương mại điện tử': ['thương mại điện tử', 'tmđt', 'ecommerce', 'e-commerce'],
        'kinh doanh quốc tế': ['business', 'international business', 'kinh doanh quốc tế'],
        'bất động sản': ['bất động sản', 'real estate', 'kinh doanh bất động sản'],
        
        # ===== Y TẾ - DƯỢC =====
        'y khoa': ['y', 'y khoa', 'y học', 'y dược', 'bác sĩ', 'y tế', 'sức khoẻ', 'sức khỏe', 'chăm sóc sức khoẻ', 'chăm sóc sức khỏe', 'khối ngành sức khoẻ', 'truyền nhiễm', 'giải phẫu', 'ngành giải phẫu', 'chuyên ngành giải phẫu', 'giải phẫu bệnh', 'giải phẫu cơ thể người', 'bsnt', 'bs đa khoa', 'đa khoa', 'kỹ thuật hình ảnh', 'kỹ thuật viên chẩn đoán', 'chuyên viên phôi học'],
        'dược': ['dược', 'dược sĩ', 'cao đẳng dược', 'dược học', 'trình dược viên', 'dược mỹ phẩm', 'retail pharmacist', 'pharmacy', 'pharmacist'],
        'điều dưỡng': ['điều dưỡng', 'nursing', 'điều dưỡng hộ lý', 'hộ lý', 'chăm sóc người già', 'elderly care', 'aged care', 'chăm sóc người cao tuổi'],
        'nha khoa': ['nha khoa', 'răng hàm mặt', 'răng–hàm–mặt', 'răng-hàm-mặt'],
        'y học cổ truyền': ['y học cổ truyền', 'yhct', 'đông y'],
        'xét nghiệm y học': ['xét nghiệm', 'kỹ thuật xét nghiệm', 'laboratory scientist'],
        'chẩn đoán hình ảnh': ['chẩn đoán hình ảnh', 'xquang', 'radiology'],
        'kỹ thuật y sinh': ['kỹ thuật y sinh', 'y sinh'],
        'vật lý trị liệu': ['vật lý trị liệu', 'phục hồi chức năng', 'phcn', 'occupational therapy', 'âm ngữ trị liệu'],
        'y tế công cộng': ['y tế công cộng', 'public health'],
        'nội khoa': ['nội khoa', 'nội tim mạch', 'tim mạch'],
        'ngoại khoa': ['ngoại khoa', 'phẫu thuật', 'surgery', 'phẫu thuật tạo hình'],
        'sản phụ khoa': ['sản phụ khoa', 'hộ sinh'],
        'nhãn khoa': ['nhãn khoa', 'mắt'],
        'da liễu': ['da liễu', 'dermatology'],
        'ung thư': ['ung thư', 'ung bướu'],
        'gây mê hồi sức': ['gây mê', 'hồi sức', 'cấp cứu'],
        'tâm lý học': ['tâm lý', 'psychology', 'tâm lý học', 'tâm lí', 'sức khoẻ tâm thần', 'psychiatry', 'chuyên ngành tâm thần', 'tư vấn trị liệu', 'tham vấn trị liệu'],
        
        # ===== TRUYỀN THÔNG - THIẾT KẾ =====
        'truyền thông': ['truyền thông', 'báo chí', 'báo chí truyền thông', 'truyền thông đa phương tiện', 'truyền thông marketing', 'nhà báo', 'báo truyền hình', 'phóng viên', 'biên tập viên', 'biên tập xuất bản', 'publishing', 'biên soạn', 'sản xuất nội dung', 'sản xuất nội dung số', 'content', 'creator', 'media life', 'phát thanh truyền hình', 'social media', 'social', 'livestream', 'writer', 'freelance'],
        'quan hệ công chúng': ['quan hệ công chúng', 'ngành quan hệ công chúng', 'học quan hệ công chúng', 'chuyên viên quan hệ công chúng', 'chuyên viên phân tích và tư vấn quan hệ công chúng', 'qhcc', 'pr', 'public relations'],
        'thiết kế đồ họa': ['thiết kế đồ hoạ', 'đồ hoạ', 'graphic design', 'designer', 'thiết kế đồ họa', 'ui', 'ux', 'ui/ux', 'layout', 'thiết kế web', 'web design', 'thiết kế sáng tạo', 'thiết kế logo', 'thiết kế logo bộ nhận diện thương hiệu', 'thiết kế trải nghiệm người dùng', 'thiết kế hệ thống'],
        'thiết kế nội thất': ['nội thất', 'thiết kế nội thất', 'interior design'],
        'thiết kế thời trang': ['thiết kế thời trang', 'thời trang', 'dệt may', 'textile', 'công nghệ may'],
        'làm phim': ['làm phim', 'dựng phim', 'biên kịch', 'quay phim', 'sân khấu điện ảnh', 'đạo diễn', 'diễn xuất', 'diễn viên', 'producer', 'hoạt hình', 'animation'],
        'âm nhạc': ['âm nhạc', 'nhạc kịch', 'ca nhạc', 'ca sĩ', 'thanh nhạc', 'nhạc sĩ', 'mc'],
        'mỹ thuật': ['mỹ thuật', 'mĩ thuật', 'nghệ thuật', 'hội hoạ', 'hoạ sĩ', 'nghệ thuật biểu diễn', 'thư pháp gia', 'múa', 'vũ công'],
        
        # ===== DU LỊCH - KHÁCH SẠN =====
        'du lịch': ['du lịch', 'quản trị du lịch', 'hướng dẫn viên du lịch', 'lữ hành', 'chuyên viên tư vấn du học', 'ngành du học và định cư'],
        'khách sạn - nhà hàng': ['khách sạn', 'nhà hàng', 'quản trị khách sạn', 'nhà hàng khách sạn', 'hospitality', 'f & b', 'dịch vụ ăn uống', 'barista', 'food blog'],
        'ẩm thực': ['bếp', 'đầu bếp', 'nấu ăn', 'chế biến món ăn', 'ẩm thực', 'làm bánh', 'chef', 'kỹ thuật chế biến món ăn', 'kỹ thuật món ăn'],
        'chăm sóc sắc đẹp': ['chăm sóc sắc đẹp', 'thẩm mỹ', 'spa', 'nail', 'làm đẹp', 'dịch vụ làm đẹp', 'chăm sóc da', 'trang điểm', 'makeup', 'tạo mẫu tóc', 'chế tạo mỹ phẩm', 'nghiên cứu mỹ phẩm'],
        'tổ chức sự kiện': ['sự kiện', 'tổ chức sự kiện', 'event management'],
        
        # ===== GIÁO DỤC - NGÔN NGỮ =====
        'sư phạm': ['sư phạm', 'sp', 'giáo dục', 'giảng viên', 'giáo viên', 'nhà giáo', 'teacher', 'primary teacher', 'secondary teacher', 'giáo sư đại học', 'tuyển sinh', 'quản lý học vụ', 'quản lý học viên', 'tư vấn học đường', 'tham vấn học đường', 'công tác hssv', 'công tác thanh thiếu niên', 'phương pháp giảng dạy'],
        'giáo dục mầm non': ['mầm non', 'ngành mầm non', 'cô giáo mầm non', 'chuyên ngành mầm non', 'gd mầm non', 'gdmn', 'early childhood', 'bachelor of early childhood', 'early childhood education'],
        'giáo dục tiểu học': ['tiểu học', 'gd tiểu học'],
        'ngôn ngữ': ['ngôn ngữ', 'ngoại ngữ', 'dịch thuật', 'biên phiên dịch', 'biên dịch', 'ngôn ngữ văn học', 'phiên dịch viên', 'tiếng anh', 'tiếng anh thương mại', 'tiếng việt', 'tiếng pháp', 'hán nôm', 'hán học', 'hán ngữ', 'song ngữ trung hàn', 'việt – nhật', 'đông nam á học', 'thái lan học', 'ngành nn trung', 'ngành nna', 'nna.anh', 'ngành ngôn ngũ'],
        'tiếng trung': ['tiếng trung', 'ngôn ngữ trung', 'ngôn ngữ trung quốc'],
        'tiếng hàn': ['tiếng hàn', 'ngôn ngữ hàn', 'ngôn ngữ hàn quốc', 'hàn quốc'],
        'tiếng nhật': ['tiếng nhật', 'nhật bản học'],
        
        # ===== LUẬT - XÃ HỘI =====
        'luật': ['luật', 'pháp luật', 'luật học', 'luật sư', 'luật kinh tế', 'luật tmqt', 'công chứng viên', 'công chứng', 'chuyên gia pháp lý kinh tế', 'chuyên viên pháp lý', 'dịch vụ pháp lý', 'tư vấn pháp lý', 'pháp chế doanh nghiệp', 'thanh tra viên', 'trợ giúp pháp lý', 'thanh tra', 'tư pháp', 'công tố viên', 'thẩm phán', 'kiểm sát viên', 'kiểm sát', 'thư ký toà án', 'thi hành án hình sự và hỗ trợ tư pháp'],
        'xã hội học': ['xã hội học', 'xã hội', 'khxh', 'công tác xã hội', 'khoa học xã hội', 'community worker', 'quan hệ cộng đồng'],
        'quan hệ quốc tế': ['quan hệ quốc tế', 'qhqt', 'international relations', 'ngoại giao', 'ngoại giao thị trường quốc tế', 'hải quan'],
        'quản lý nhà nước': ['quản lý nhà nước', 'hành chính nhà nước', 'quản lý công', 'hành chính công', 'public administration', 'ql nhà nước', 'chính sách công', 'nhà phân tích chính sách'],
        'chính trị học': ['chính trị', 'chính trị học', 'chính trị gia', 'tư tưởng hồ chí minh'],
        'triết học': ['triết học', 'ngành triết học'],
        
        # ===== KHOA HỌC CƠ BẢN =====
        'toán học': ['toán ứng dụng', 'ngành toán học', 'toán học', 'toán tin', 'ngành toán - tin', 'thống kê', 'chuyên viên thống kê'],
        'vật lý': ['ngành vật lý học', 'vật lý học', 'vật lý', 'vật lý kỹ thuật', 'vật lý nguyên tử và hạt nhân', 'kỹ sư hạt nhân', 'kỹ thuật hạt nhân', 'khoa học vật lý', 'môn vật lý', 'khí tượng thuỷ văn'],
        'hóa học': ['hoá học', 'hóa học', 'chemistry', 'kỹ thuật hoá học', 'hoá dầu'],
        'sinh học': ['công nghệ sinh học', 'sinh học', 'biotech', 'kỹ thuật sinh học'],
        'thiên văn học': ['thiên văn'],
        'địa chất': ['mỏ địa chất', 'địa chất', 'địa chất học', 'địa chất học đại học', 'ngành học địa chất', 'kỹ thuật địa chất', 'ngành địa chất', 'dầu khí', 'ngành sản xuất dầu khí và than', 'kỹ thuật tuyển khoáng'],
        'lịch sử': ['lịch sử', 'history', 'sư phạm lịch sử', 'lịch sử đảng'],
        'địa lý': ['địa lý', 'geography'],
        'văn học': ['ngữ văn', 'sp ngữ văn', 'văn học', 'sư phạm ngữ văn'],
        'văn hóa - nghệ thuật': ['nhân văn', 'văn hoá', 'quản lý văn hoá', 'văn hoá thể thao'],
        
        # ===== NÔNG - LÂM - THUỶ SẢN =====
        'nông nghiệp': ['nông nghiệp', 'nông lâm', 'chăn nuôi', 'nông học', 'bảo vệ thực vật', 'hoa cây cảnh', 'công nghệ rau , hoa quả và cảnh quan', 'lâm nghiệp', 'forestry'],
        'thuỷ sản': ['nuôi trồng thuỷ sản', 'thuỷ sản', 'công nghệ nuôi trồng thuỷ sản', 'bệnh học thuỷ sản', 'công nghệ chế biến thuỷ sản'],
        'thú y': ['thú y', 'bác sĩ thú y'],
        'môi trường': ['môi trường', 'công nghệ môi trường', 'công nghệ xử lý nước thải', 'tài nguyên', 'phát triển bền vững', 'sustainable development'],
        'công nghệ thực phẩm': ['công nghệ thực phẩm', 'cntp', 'food technology', 'thực phẩm', 'hoá thực phẩm', 'dinh dưỡng', 'nutrition'],
        
        # ===== HÀNG KHÔNG - HÀNG HẢI =====
        'hàng không': ['phi công', 'hàng không', 'tiếp viên hàng không', 'aerospace', 'hàng không vũ trụ', 'công nghệ hàng không', 'quản lý và khai thác bay', 'quản lý hoạt động bay', 'khai thác vận tải', 'phi hành gia', 'thiết bị bay không người lái', 'uav', 'drone'],
        'hàng hải': ['tàu biển', 'tàu thủy', 'lái tàu', 'kỹ thuật tàu thuỷ', 'quản lý tàu biển', 'hàng hải', 'khoa học hàng hải', 'quản lí hàng hải', 'hậu cần và quản lý hàng hải', 'vận tải và hậu cần', 'thuyền trưởng', 'thuyền phó', 'thuỷ thủ', 'sĩ quan boong', 'đại phó', 'chuyên gia tư vấn an toàn tàu', 'đi biển', 'tàu ngầm', 'đăng kiểm'],
        'hải dương học': ['hải dương học', 'ngành hải dương học', 'kỹ thuật biển', 'thiết kế tàu và công trình ngoài khơi'],
        
        # ===== AN NINH - QUỐC PHÒNG =====
        'an ninh - quốc phòng': ['quân sự', 'bộ đội', 'military', 'công an', 'cảnh sát', 'quân đội', 'quân nhân', 'an ninh quốc phòng', 'giáo dục quốc phòng', 'gdqpan', 'an ninh quốc gia', 'nghiệp vụ an ninh – điều tra', 'nghiệp vụ an ninh', 'điều tra hình sự', 'phòng chống tội phạm', 'sĩ quan', 'lực lượng vũ trang', 'phòng không không quân', 'sĩ quan chỉ huy tham mưu không quân', 'biên phòng', 'xuất nhập cảnh', 'quản lý trật tự an toàn giao thông', 'bảo vệ', 'nghiệp vụ xuất nhập cảnh', 'immigration', 'lính cứu hoả', 'cứu hỏa', 'firefighter', 'pccc'],
        
        # ===== NGÀNH KHÁC =====
        'thông tin - thư viện': ['thông tin - thư viện', 'ngành thông tin - thư viện', 'thư viện thông tin', 'lưu trữ học', 'nhân viên thư viện', 'ngành thư viện thông tin'],
        'đông phương học': ['đông phương học', 'ngành đông phương học'],
        'việt nam học': ['việt nam học', 'ngành việt nam học', 'văn hoá việt nam', 'văn hoá dân tộc thiểu số việt nam'],
        'khảo cổ học': ['chuyên ngành khảo cổ học', 'khảo cổ học'],
        'tôn giáo học': ['ngành tôn giáo học', 'tôn giáo học'],
        'thể dục thể thao': ['ngành gdtc', 'thể dục thể thao', 'thể dục', 'thể thao', 'huấn luyện thể thao'],
        'quản lý chất lượng': ['quản lý chất lượng', 'chuẩn mực', 'quản lý kỹ thuật']
    }
    
    name = normalized_name.lower().strip()
    if not name:
        return None
    
    # Bước 1: Exact match với key
    if name in major_mapping:
        return name
    
    # Bước 2: Exact match với variants
    for key, variants in major_mapping.items():
        if name in variants:
            return key
    
    # Bước 3: Substring match CẢI TIẾN - CHỈ match nếu variant >=3 ký tự
    for key, variants in major_mapping.items():
        for variant in variants:
            if len(variant) >= 3:
                if variant in name or name in variant:
                    return key
    
    # Bước 4: Word overlap 80%
    name_words = set(name.split())
    if len(name_words) >= 1:
        for key, variants in major_mapping.items():
            for variant in variants:
                variant_words = set(variant.split())
                if len(variant_words) == 0:
                    continue
                overlap = len(variant_words & name_words)
                if overlap / len(variant_words) >= 0.8 or overlap == len(variant_words):
                    return key
    
    # KHÔNG TÌM THẤY trong dictionary → trả về None
    return None

## 4. Đọc dữ liệu từ bảng Post_Entity (chỉ entityType = 'MAJ')

In [ ]:
# Đọc dữ liệu từ bảng Post_Entity với JOIN Entity để lọc chỉ entityType = 'MAJ'
df_post_entity = spark.sql("""
    SELECT 
        pe.postID,
        pe.entityID,
        pe.entityOrder,
        pe.entityName,
        pe.entityNameNormalized,
        pe.created_at,
        pe.updated_at,
        e.entityType
    FROM nessie.gold_result_model_multi_task.Post_Entity pe
    INNER JOIN nessie.gold_result_model_multi_task.Entity e
        ON pe.entityID = e.entityID
    WHERE e.entityType = 'MAJ'
""")

total_count = df_post_entity.count()
print(f"✓ Đã đọc {total_count:,} records từ Post_Entity (chỉ entityType = 'MAJ')")
df_post_entity.show(10, truncate=False)

## 5. Tạo UDF và áp dụng mapping

In [ ]:
# Tạo UDF và áp dụng mapping
from pyspark.sql.functions import when

norm_udf = udf(normalize_text, StringType())
map_udf = udf(find_major_group, StringType())

# LOGIC MỚI: Lưu TẤT CẢ mapped values (không so sánh với clean nữa)
df_mapped = df_post_entity \
    .withColumn("clean", norm_udf(col("entityName"))) \
    .withColumn("mapped", map_udf(col("clean"))) \
    .withColumn("entityNameNormalized_new", col("mapped"))

print("✓ Đã áp dụng mapping (lưu TẤT CẢ mapped values)")
df_mapped.select("entityName", "clean", "mapped", "entityNameNormalized_new") \
    .show(20, truncate=False)

df_mapped.cache()
df_post_entity.unpersist()
print("✓ Đã cache df_mapped")


## 6. Thống kê kết quả mapping

In [ ]:
# Thống kê kết quả
total = df_mapped.count()
mapped = df_mapped.filter(col("entityNameNormalized_new").isNotNull()).count()
unmapped = total - mapped

print("\n" + "="*70)
print(" KẾT QUẢ MAPPING")
print("="*70)
print(f"Tổng: {total:,} records")
print(f"✓ Mapped: {mapped:,} ({mapped/total*100:.2f}%)")
print(f"✗ Unmapped: {unmapped:,} ({unmapped/total*100:.2f}%)")

# Top nhóm
print("\n TOP 20 NHÓM NGÀNH:")
df_mapped.filter(col("entityNameNormalized_new").isNotNull()) \
    .groupBy("entityNameNormalized_new").count() \
    .orderBy(col("count").desc()) \
    .show(20, truncate=False)

# Top chưa map
if unmapped > 0:
    print("\n TOP 15 CHƯA MAP:")
    df_mapped.filter(col("entityNameNormalized_new").isNull()) \
        .groupBy("entityName").count() \
        .orderBy(col("count").desc()) \
        .show(15, truncate=False)


## 7. Cập nhật cột entityNameNormalized trong bảng Post_Entity

In [ ]:
# Chuẩn bị và thực hiện UPDATE
from pyspark.sql.functions import current_timestamp

df_update = df_mapped.select(
    "postID", "entityID", "entityOrder", "entityName",
    col("entityNameNormalized_new").alias("entityNameNormalized"),
    "created_at", current_timestamp().alias("updated_at")
)

df_update.cache()
count = df_update.count()
print(f"  Updating {count:,} records...")

df_update.createOrReplaceTempView("temp_updates")

spark.sql("""
    MERGE INTO nessie.gold_result_model_multi_task.Post_Entity AS t
    USING temp_updates AS s
    ON t.postID = s.postID AND t.entityID = s.entityID AND t.entityOrder = s.entityOrder
    WHEN MATCHED THEN UPDATE SET
        t.entityNameNormalized = s.entityNameNormalized,
        t.updated_at = s.updated_at
""")

print(" Update completed!")

spark.catalog.dropTempView("temp_updates")
df_update.unpersist()
df_mapped.unpersist()
print("✓ Cleanup done")


## 8. Dừng Spark Session

In [ ]:
# Cleanup và dừng Spark
import gc

spark.catalog.clearCache()
gc.collect()
spark.stop()

print(" Spark Session stopped!")
